In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

In [ ]:
# モデルの共通部分となるクラス
# すべての引数はただのリスト。クラスの初期化時に""np.reshape()""を実行する
class ModelBase2:
    def __init__(self, train_x, train_y, target_x, target_y, benchmark_name="benchmark_name", function_name="function_name"):
        self.benchmark_name = benchmark_name
        self.function_name = function_name
        
        self.train_x  = np.reshape(train_x, (-1, 1))
        self.train_y  = np.reshape(train_y, (-1, 1))
        self.target_x = np.reshape(target_x, (-1, 1))
        self.target_x = np.reshape(target_x, (-1, 1))

# 線形モデルでロバスト回帰を行う
# 作成したModelBase2を継承
class ModelLin_rob(ModelBase2):
    def calc_hr(self):
        self.hr = HuberRegressor()
        self.hr.fit(self.train_x, self.train_y)

    def calc_mape_score(self):
        test_y_predicted = self.lr.predict(self.target_x)
        self.mape_score = float(mape_score(self.target_y, test_y_predicted))

    def calc_mape_score_InTrain(self):
        train_y_predicted = self.lr.predict(self.train_x)
        self.mape_score_InTrain = float(mape_score(self.train_y, train_y_predicted))

    def predict(self, num):
        predicted = self.hr.predict(num)
        return(predicted)
    
    def ModelName(self):
        return("ModelLin_rob")

In [ ]:
FixedClassDict = {}
for benchmark in benchmarks:
    targetDF = return_ErrorRateFixedClassDF(benchmark=benchmark, FixClass="B", predict_process=256)
    columns = targetDF.columns.tolist()
    # 平均絶対誤差率が0.1より大きいものを抽出
    targetDF = targetDF[targetDF[columns[0]] > 0.1]
    # データが同じことに由来する重複を削除
    targetDF = targetDF[~targetDF.duplicated()]
    targetFunctionNameList = targetDF.index.tolist()
    FixedClassDict[benchmark] = targetFunctionNameList
FixedClassDict

# Extra-Pへの入力ファイルを生成
funcCallNumDict = {}
for benchmarkName in FixedClassDict:
    if benchmarkName == "bt" or benchmarkName == "sp" :
        _processes = process_onlyBTSP
    else:
        _processes = process_excludeBTSP
    funcCallNumDict[benchmarkName] = {}
    for functionName in FixedClassDict[benchmarkName]:
        funcCallNumDict[benchmarkName][functionName] = returnRawDF(Benchmark = benchmarkName, functionName = functionName, fix="Class", Processes = _processes, FixedBenchmarkClass="B")
#         generateInputFileForExtraP(funcCallNumDict[benchmarkName][functionName], repeated=3, includeLastData = False, fileName = f"tmp_{benchmarkName}_{functionName}.txt")

plt.figure(figsize=(10,10),dpi=200)


# ベンチマーク名と関数名の指定
benchmarkName = "ft"
functionName = "CFFTZ"
print(f"___________________\"{benchmarkName}\"_____\"{functionName}\"___________________")

# 実データの取り出し
functionDF = funcCallNumDict[benchmarkName][functionName]
functionDF
raw_x = functionDF.columns.tolist()
raw_y = [functionDF.at[functionName, x] for x in raw_x]

# Extra-P用の入力データの作成
generateInputFileForExtraPfromLists(input_x=raw_x, input_y=raw_y, repeated=3, includeFirstData=False, includeLastData=False, fileName =f"tmp_{benchmarkName}_{functionName}.txt")
raw_x = np.array(raw_x).reshape(-1,1)
raw_y = np.array(raw_y).reshape(-1, 1)
# Extra-PでfixProcessデータを入力して出力したモデルの図示
plot_x = np.linspace(0.01, 256, 500)
# 下記はExtra-Pで出力されたモデル
# 192658.33552658802 + -46041.48912003825 * p^(1/3)
plot_y = []
plot_y = [192658.33552658802 + -46041.48912003825 * x**(1/3) for x in plot_x]
plt.plot(plot_x, plot_y, label="ExtraP")

# 説明変数と目的変数に分割
notTrain_x, train_x, target_x = raw_x[0], raw_x[1:-1], raw_x[-1]
notTrain_y, train_y, target_y = raw_y[0], raw_y[1:-1], raw_y[-1]

plt.scatter(train_x, train_y, marker="o", label="予測に用いた関数コール回数")
plt.scatter(target_x, target_y, marker="o", label="予測したい関数コール回数の実測値")
plt.scatter(notTrain_x, notTrain_y, marker="o", label="予測に用いなかった関数コール回数")

# モデル式をプロットするために変数”plot_x”に対して変換を実施
plot_x = np.array(plot_x).reshape(-1, 1)

# 線形モデル
model_lin = ModelLin(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_lin.calc_lr()
plot_y_lin = model_lin.predict(plot_x)
plt.plot(plot_x, plot_y_lin, label="線形モデル(最初のデータを除外)")
# 対数モデル
model_log10 = ModelLog10(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_log10.calc_lr()
plot_y_log10 = model_log10.predict(plot_x)
plt.plot(plot_x, plot_y_log10, label="対数モデル(最初のデータを除外)")
# 反比例モデル
model_ip = ModelIP(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_ip.calc_lr()
plot_y_ip = model_ip.predict(plot_x)
plt.plot(plot_x, plot_y_ip, label="反比例モデル(最初のデータを除外)")
# 線形飽和モデル
model_branch = ModelBranch(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_branch.calc_lr()
plot_y_branch = model_branch.predict(plot_x)
plt.plot(plot_x, plot_y_branch, label="線形飽和モデル(最初のデータを除外)")

# 最初のデータを含む学習データと最後のデータのみの試験用データに分割
train_x, target_x = raw_x[:-1], raw_x[-1]
train_y, target_y = raw_y[:-1], raw_y[-1]
raw_x
train_x
target_x
raw_y
train_y
target_y

# 線形モデル(ロバスト回帰)
model_lin_rob = ModelLin_rob(train_x=train_x, train_y=train_y, target_x=target_x, target_y=target_y)
model_lin_rob.calc_hr()
plot_y_lin_rob = model_lin_rob.predict(plot_x)
plt.plot(plot_x, plot_y_lin_rob, label="線形モデル（ロバスト回帰）")

# 凡例の表示
plt.legend()
# 軸ラベルの設定
plt.ylabel("関数コール回数")
plt.xlabel("実行コア数")
# 軸の対数軸化
# plt.xscale('log')
plt.yscale('log')